In [1]:
import networkx as nx
import preprocessing as prep
import sys  
import datetime
import pytz
import numpy as np
import pandas as pd

from multiprocessing import Pool
from bb_binary import FrameContainer, Repository, load_frame_container
from collections import namedtuple 
from pandas import DataFrame, Series

import pipeline_frames_parallel as pfp

In [2]:
startT = "2016-07-21T15:00:00Z"
startT_dt = datetime.datetime.strptime(startT, "%Y-%m-%dT%H:%M:%SZ").replace(tzinfo=pytz.UTC)
startT_ts = startT_dt.timestamp()
endT_dt = startT_dt + datetime.timedelta(minutes=1)
endT_ts = endT_dt.timestamp()

In [3]:
enu = 0
path = "/mnt/data2016/"
b = startT_ts
e = endT_ts
confidence = 0.95
distance = 212
ilen = 3
year = 2016
gap = 2

In [ ]:
def generate_network(enu, path, b, e, confidence, distance, ilen, year, gap):
    
    repo = Repository(path)
    xmax = 3000
    offset = 200
    
    Detection = namedtuple('Detection',
                           ['idx', 'xpos', 'ypos', 'radius', 'zRotation', 'decodedId', 'frame_idx', 'timestamp', 'cam_id', 'fc_id'])
    # one df per cam
    parts = np.empty(4, dtype=object)


    for i in list(range(4)):
         
        tpls = []
        myid = 0


        for frame, fc in repo.iter_frames(begin=b, end=e, cam=i):
            for d in frame.detectionsUnion.detectionsDP:
                d = Detection(d.idx, d.xpos, d.ypos, d.radius, d.zRotation, list(d.decodedId), myid, frame.timestamp, fc.camId, fc.id)
                tpls.append(d)
            myid += 1
        
        df = DataFrame(tpls)
        print("#{} DF-{}: {}, {}, {}".format(enu, i, df.shape, datetime.datetime.fromtimestamp(b, tz=pytz.UTC),datetime.datetime.fromtimestamp(e, tz=pytz.UTC)))
        df = prep.calcIds(df, confidence, year)
        parts[i] = df
    
    if year == 2015:
        # cam 0 und cam1 nach rechts verschieben
        parts[0].xpos = parts[0].xpos + xmax + offset
        parts[1].xpos = parts[1].xpos + xmax + offset

        # Seiten zusammenfugen
        side0 = pd.concat([parts[3], parts[0]])
        side1 = pd.concat([parts[2], parts[1]])

    if year == 2016:
        # cam 1 und cam 3 nach rechts verschieben
        parts[1].xpos = parts[1].xpos + xmax + offset
        parts[3].xpos = parts[3].xpos + xmax + offset

        # Seiten zusammenfugen
        side0 = pd.concat([parts[0], parts[1]])
        side1 = pd.concat([parts[2], parts[3]])


    close1 = prep.get_close_bees_ckd(side0, distance)
    close2 = prep.get_close_bees_ckd(side1, distance)

    close = pd.concat([close1,close2])

    p = prep.bee_pairs_to_timeseries(close)

    # Coorect pair time series
    p_corrected = p.apply(prep.fill_gaps, axis=1, args=[gap])

    return prep.extract_interactions(p_corrected,ilen)

In [ ]:
edges = []
for i in [1,2,3]:
    edges.append(generate_network(enu,path,b,e,confidence,distance,ilen,year,gap))
    b = e
    e = e + 60

In [ ]:
E = pd.concat(edges)

In [ ]:
G = nx.Graph()

In [ ]:
edges = DataFrame(E.tolist(), columns=['frequency', 'totalduration', 'durations'], index=E.index).reset_index().rename(columns={'index':'pair'})

In [ ]:
edges.head()

In [ ]:
pairs = edges.groupby(by="pair")

In [ ]:
def myfunction(zeugs):
    retval = str(sum(zeugs.values, []))
    return retval

In [ ]:
ss = Series([[1,2],[3,4],[2,3]])

In [ ]:
myfunction(ss)

In [ ]:
summ = pairs.aggregate({'frequency': sum, 'totalduration': sum, 'durations': myfunction}).reset_index()

In [ ]:
for index, x in summ.iterrows():
    G.add_edge(x.pair[0], x.pair[1], frequency=x.frequency, totalduration=x.totalduration, durations=x.durations)



In [ ]:
print(nx.info(G))

In [4]:
edges = pfp.generate_network(enu,path,b,e,confidence,distance,ilen,year,gap)

#0 DF-0: (22026, 10), 2016-07-21 15:00:00+00:00, 2016-07-21 15:01:00+00:00
#0 DF-1: (25064, 10), 2016-07-21 15:00:00+00:00, 2016-07-21 15:01:00+00:00
#0 DF-2: (18500, 10), 2016-07-21 15:00:00+00:00, 2016-07-21 15:01:00+00:00
#0 DF-3: (23292, 10), 2016-07-21 15:00:00+00:00, 2016-07-21 15:01:00+00:00


In [5]:
G = prep.create_graph2(edges)

In [6]:
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 598
Number of edges: 2285
Average degree:   7.6421


In [8]:
G.get_edge_data(20,1068)

{'durations': '(25,)', 'frequency': 1, 'totalduration': 25}

In [ ]:
(str([1,2,3]))

In [9]:
nx.write_graphml(G, "ttt" + ".graphml")
